# Segmenting and Clustering Neighborhoods in Toronto

## Data collection

### 1. Webscraping

In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

In [4]:
## Get html data from wikipedia url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)

In [6]:
## Create beautifulsoup object with html.parser as parameter
soup = BeautifulSoup(html_data.content, 'html.parser')

In [173]:
## Find table and table body. Print table body
table = soup.find('table')
table_body = table.find('tbody')
print(table_body.prettify())

<tbody>
 <tr>
  <td style="width:11%; vertical-align:top; color:#ccc;">
   <p>
    <b>
     M1A
    </b>
    <br/>
    <span style="font-size:85%;">
     <i>
      Not assigned
     </i>
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top; color:#ccc;">
   <p>
    <b>
     M2A
    </b>
    <br/>
    <span style="font-size:85%;">
     <i>
      Not assigned
     </i>
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top;">
   <p>
    <b>
     M3A
    </b>
    <br/>
    <span style="font-size:85%;">
     <a href="/wiki/North_York" title="North York">
      North York
     </a>
     <br/>
     (
     <a href="/wiki/Parkwoods" title="Parkwoods">
      Parkwoods
     </a>
     )
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top;">
   <p>
    <b>
     M4A
    </b>
    <br/>
    <span style="font-size:85%;">
     <a href="/wiki/North_York" title="North York">
      North York
     </a>
     <br/>
     (
     <a href="/wiki/Victoria_Vill

In [215]:
## Parse through body to find postal code, borough and neighborhood
tr = table_body.find_all('td') # information located in the td tags
table_contents=[]
for rows in tr:
    code = rows.find('b')
    cell={}
    if rows.span.text == 'Not assigned':
        pass
    else:
        title = rows.span.text
        cell['Postal Code']=code.text
        cell['Borough'] = title.split('(')[0]
        cell['Neighborhood'] = title.split('(')[1].strip(')').replace(' /', ',')
        table_contents.append(cell)

In [216]:
# Create dataframe of data
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [221]:
# View dataframe
df.head(11)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills)North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [222]:
df.shape

(103, 3)

### 2. Find longitude and latitude coordinates for each postal code

In [180]:
!pip install geocoder

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491098 sha256=b2b6b01d74051674b0a8a8d25d784d179f71bda6bab3b907b67535ff58a47261
  Stored in directory: c:\users\krishant\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future


In [208]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = 'M9L'
# loop until you get the coordinates
while(lat_lng_coords is None): 
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    print('hello')
    


latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


KeyboardInterrupt: 

In [185]:
from geopy.geocoders import Nominatim

In [207]:
address = 'M9L, Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [192]:
## Previous methods are not consistent, so I downloaded the longitude and latitude data for the neighborhoods
## and appended the borough and neighborhoods

In [213]:
df_lnglat = pd.read_csv('Geospatial_Coordinates.csv')

In [214]:
df_lnglat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [218]:
df_hood = pd.merge(df, df_lnglat, on='Postal Code')

In [219]:
df_hood

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Exploration and Clustering

#### In this section we explore all the most common venues of each neighborhood in Toronto, Ontario. We then cluster the different boroughs by common venues via K-Means clustering.

In [223]:
import folium

In [226]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_hood['Latitude'], df_hood['Longitude'], df_hood['Borough'], df_hood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [282]:
CLIENT_ID = 'W1K3B3EXHOUCBDHAI5DIIXAP5Y0OCHFHMXGLVAXNZOFX4YCS' # your Foursquare ID
CLIENT_SECRET = '3KZ5JW1MQZWV531ZKJOGWULIWA1SOZXL2GH1HHL1M0RXZFQX' # your Foursquare Secret
ACCESS_TOKEN = 'YYCIEGWGT4TM1AGLAVMAJ2WZUKQDVEWGCFKJVRVF0LYKSPGF' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1K3B3EXHOUCBDHAI5DIIXAP5Y0OCHFHMXGLVAXNZOFX4YCS
CLIENT_SECRET:3KZ5JW1MQZWV531ZKJOGWULIWA1SOZXL2GH1HHL1M0RXZFQX


#### Function to retrieve 100 venues within a radius of 500 from the neighborhoods

In [228]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Apply function and store data in toronto_venues

In [229]:
toronto_venues = getNearbyVenues(names=df_hood['Neighborhood'],
                                   latitudes=df_hood['Latitude'],
                                   longitudes=df_hood['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills)North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills)South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview)East  
The Danfor

In [230]:
print(toronto_venues.shape)
toronto_venues.head()

(2122, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### Count of how many venues in each neighborhood

In [231]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt),5,5,5,5,5,5
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,4,4,4,4,4,4
Woodbine Heights,5,5,5,5,5,5


In [232]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


#### Pre-processing data for clustering

In [233]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
toronto_onehot.shape

(2122, 272)

#### Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [235]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
toronto_grouped.shape

(100, 272)

#### Print each neighborhood with the top 5 venues

In [237]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt) ----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2     Coffee Shop  0.17
3             Pub  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.09
1         Bank  0.09
2    Pet Store  0.05
3  Gas Station  0.05
4         Park  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4   Indian Restaurant  0.

----North Toronto West----
                 venue  freq
0       Clothing Store  0.14
1          Coffee Shop  0.10
2          Yoga Studio  0.05
3  Sporting Goods Shop  0.05
4                 Park  0.05


----Northwood Park, York University----
                   venue  freq
0         Massage Studio   0.2
1                    Bar   0.2
2            Coffee Shop   0.2
3  Vietnamese Restaurant   0.2
4   Caribbean Restaurant   0.2


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Ontario Provincial Government----
                venue  freq
0         Coffee Shop  0.18
1    Sushi Restaurant  0.06
2         Yoga Studio  0.03
3  Italian Restaurant  0.03


#### Function to sort the venues in descending order

In [238]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Put the top 10 venues of each neighborhood in a dataframe

In [241]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt),Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Pub,Coffee Shop,German Restaurant,Drugstore,Doner Restaurant,Dog Run,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Fried Chicken Joint,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Shopping Mall,Restaurant,Deli / Bodega
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Liquor Store,Thai Restaurant,Fast Food Restaurant,Butcher,Pub,Café


## K-Means Clustering

In [242]:
from sklearn.cluster import KMeans

In [259]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [264]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', int(kmeans.labels_))

toronto_merged = df_hood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Restaurant,Pub,Café,Theater,Spa,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Carpet Store,Discount Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Sandwich Place,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1.0,Coffee Shop,Bakery,Restaurant,Italian Restaurant,Café,Pizza Place,Pub,Indian Restaurant,Diner,Bank
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1.0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,Steakhouse,American Restaurant,Seafood Restaurant
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3.0,River,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Dance Studio


#### Drop any missing values

In [272]:
toronto_merged.dropna(axis=0,inplace=True)

In [273]:
toronto_merged.head(6)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Restaurant,Pub,Café,Theater,Spa,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Carpet Store,Discount Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Sandwich Place,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Print Shop,Fast Food Restaurant,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


#### Explore map of clusters

In [274]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Clearly from the dataframe below, cluster's 0 are (red circles) related to venues that are parks

In [275]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Food & Drink Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
14,East York,0.0,Park,Curling Ice,Beer Store,Skating Rink,Intersection,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
21,York,0.0,Park,Women's Store,Pool,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
35,East York/East Toronto,0.0,Pizza Place,Park,Convenience Store,Metro Station,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
40,North York,0.0,Bus Stop,Park,Other Repair Shop,Airport,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
46,North York,0.0,Grocery Store,Bank,Shopping Mall,Park,Gluten-free Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Gourmet Shop
49,North York,0.0,Construction & Landscaping,Basketball Court,Park,Bakery,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
52,North York,0.0,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
61,Central Toronto,0.0,Park,Bus Line,Swim School,Business Service,College Stadium,College Rec Center,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
64,York,0.0,Park,Convenience Store,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


#### Cluster 1 is the most common cluster and includes a variety of shops, from a glance there seems to be a lot of coffee shops

In [276]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Restaurant,Pub,Café,Theater,Spa,Shoe Store
3,North York,1.0,Clothing Store,Furniture / Home Store,Women's Store,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Carpet Store,Discount Store
4,Queen's Park,1.0,Coffee Shop,Sushi Restaurant,Gym,Distribution Center,Sandwich Place,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant
6,Scarborough,1.0,Print Shop,Fast Food Restaurant,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1.0,Coffee Shop,Bakery,Restaurant,Italian Restaurant,Café,Pizza Place,Pub,Indian Restaurant,Diner,Bank
97,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,Steakhouse,American Restaurant,Seafood Restaurant
99,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Fast Food Restaurant,Hotel,Mediterranean Restaurant
100,East Toronto Business,1.0,Light Rail Station,Brewery,Recording Studio,Skate Park,Burrito Place,Fast Food Restaurant,Farmers Market,Spa,Comic Shop,Restaurant


#### Cluster 2 only have 3 venues. This cluster classifies the North York and Etobicoke boroughs as baseball fields and womens store

In [277]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,2.0,Food Truck,Baseball Field,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
57,North York,2.0,Baseball Field,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Ethiopian Restaurant
101,Etobicoke,2.0,Baseball Field,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Ethiopian Restaurant


#### Cluster 3 is related to the venue river

In [278]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,3.0,River,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Dance Studio


#### Cluster 4 classifies the Scarborough borough has having motel as the most common venue

In [279]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Scarborough,4.0,Motel,American Restaurant,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
